In [267]:
import pandas as pd
import numpy as np
from sklearn import metrics
from data_analysis import get_start, get_start_end

In [268]:
filepath = "../Data/Combined/FHD_55.csv"
data = pd.read_csv(filepath, index_col=False)

In [269]:
#for val in data:
#    if val != 'time':
#        data[val] = (data[val] - min(data[val])) / (max(data[val]) - min(data[val]))

In [270]:
data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

In [271]:
idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

In [272]:
mx, idx_max

(10.075528058846395, 103)

In [273]:
filepath = "../Data/Combined/SRV_50.csv"
data = pd.read_csv(filepath, index_col=False)

data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)
data['controller_right_rel_headset'] = (data['headset_pos.z']) - (data['controller_right_pos.z'])


idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

print(data['controller_right_vel.y'][idx_max])
print(data['controller_right_vel.x'][idx_max])
print(data['controller_left_vel.z'][idx_max])

-11.74106
0.1610769
-2.119427


In [274]:
def simple_stat_classifier(filepath):
    data = pd.read_csv(filepath, index_col=False)
    data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

    idx_max = -1
    max = -float('inf')
    for i in range(len(data['controller_right_vel'])):
        if float(data['controller_right_vel'][i]) > max:
            max = float(data['controller_right_vel'][i])
            idx_max = i

    if float(data['controller_right_vel.y'][idx_max]) > 0:
        # BHD or FHD
        if float(data['controller_right_vel.x'][idx_max]) < 0:
            return "FHD"
        else:
            return "BHD"
    else:
        # SRV or VOL
        if float(data['controller_right_vel.y'][idx_max]) < -3.5: # Really wouldn't want to use this part...
            return "SRV"
        else:
            return "VOL"

In [275]:
simple_stat_classifier("../Data/Combined/BHD_13.csv")

'BHD'

Preliminary Data Analysis seems to show that if we just take the controller_right_vel.y datapoint at the highest controller velocity, we can differentiate between serves and FHD/BHD/VOL. 

SRV is highly negative. (controller moving downwards)

FHD, BHD, are highly positive. (controller moving upwards)

VOL is slightly negative. (controller moving downwards)

Then out of FHD, BHD, we can check controller_right_vel.x
 - If > 0, BHD. (Controller moving to the right)
 - If < 0, FHD. (Controller moving to the left)

But magnitude is dependent on person, so we might not be able to use that... Might only be able to use +/-

In [276]:
def test_accuracy(classifier, errors=False):
    """
    Function to test how accurate your classifier is. Takes a classifier 
    (a function) that takes a filepath, and returns a string, either 
    'BHD', 'FHD', 'VOL', or 'SRV'. 

    Also can specifiy whether to print errors.

    Returns accuracy score of the classifier. 
    """
    actual = []
    predicted = []
    csv_number = []
    for action in ['BHD','FHD','VOL','SRV']:
        for i in range(1,91):
            filepath = "../Data/Combined/" + action + "_" + str(i).zfill(2) + ".csv"
            actual.append(action)
            predicted.append(classifier(filepath))
            csv_number.append(i)
    
    if errors:
        for i in range(len(actual)):
            if actual[i] != predicted[i]:
                print(actual[i], predicted[i], csv_number[i])
    
    return metrics.accuracy_score(actual, predicted)

In [277]:
def simple_stat_classifier_2(filepath):
    """
    Takes a filepath containing a sensor trace. First trims data according to
    isolate the swing action. Then, based on trimmed csv file, runs statistical
    classifier to determine swing type.

    Returns string, determining guessed swing type.
    """
    data = pd.read_csv(filepath, index_col=False)
    data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

    start, end = get_start_end(data) # Determine start and end of swing
    data = data[start:end]
    
    idx_max = -1
    mx = -float('inf') # Finding moment of maximum velocity
    for i in range(len(data['controller_right_vel'])):
        i += start
        if float(data['controller_right_vel'][i]) > mx:
            mx = float(data['controller_right_vel'][i])
            idx_max = i

    if float(data['controller_right_vel.y'][idx_max]) > 0: # If swing moves upwards
        # BHD or FHD
        if float(data['controller_right_vel.x'][idx_max]) < 0: # If swing moves left/right
            return "FHD"
        else:
            return "BHD"
    else:
        # SRV or VOL    # Finding peak of controller, relative to headset
        if max((data['controller_right_pos.y'] - data['headset_pos.y'])) > 0.2: # 0.2 may need to be normalized
        #if max((data['controller_right_pos.y'] - data['headset_pos.y'])) > 0: # This works for Ian/Lucien, but gets Jason wrong every single time lol
            return "SRV"
        else:
            return "VOL"

TESTING BELOW

In [278]:
filepath = "../Data/Combined/SRV_57.csv"
data = pd.read_csv(filepath, index_col=False)

data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

In [279]:
s = get_start(data)
data['headset_pos.y'][s] - data['controller_right_pos.y'][s]

-0.40462167

In [280]:
data['headset_pos.y'].mean()

0.015195051647058843

In [281]:
test_accuracy(simple_stat_classifier)

0.9805555555555555

In [284]:
test_accuracy(simple_stat_classifier_2)

0.9944444444444445

In [312]:
def get_start_end_after_classification(data, swing):
    """
    Another preliminary Function to determine start and end indices for a given FHD
    or BHD sensor trace.

    Returns tuple, of indices at which start and end of swing are estimated
    to be.
    """
    start = 0
    end = len(data["controller_right_vel.z"])

    data["controller_right_vel"] = (
        data["controller_right_vel.x"] ** 2
        + data["controller_right_vel.y"] ** 2
        + data["controller_right_vel.z"] ** 2
    ) ** (1 / 2)

    pos = -1
    neg = -1
    min = float("inf")
    max = -float("inf")
    apex = -1
    max_apex = -float("inf")

    for i in range(
        len(data["controller_right_vel"])
    ):  # Finding moment of highest Velocity
        if float(data["controller_right_vel"][i]) > max_apex:
            apex = i
            max_apex = data["controller_right_vel"][i]

    for i in range(apex):
        if (
            float(data["controller_right_vel.z"][i]) > max
        ):  # Finding moment of highest velocity forwards, up to apex
            max = float(data["controller_right_vel.z"][i])
            pos = i
    for i in range(
        apex, len(data["controller_right_vel.z"])
    ):  # Finding moment of highest velocity backwards, after apex
        if float(data["controller_right_vel.z"][i]) < min:
            min = float(data["controller_right_vel.z"][i])
            neg = i

    for i in range(pos, 0, -1):  
        # Search backwards from moment of highest velocity forwards. Finds moment when z-velocity becomes significant.
        if (
            data["controller_right_vel.z"][i] >= 0.2
            and data["controller_right_vel.z"][i - 1] < 0.2
        ):
            start = i
            break

    if (
        swing != 'VOL'
    ):  
        for i in range(
            neg, len(data["controller_right_vel.z"]) - 1
        ):  # Search forwards from moment of highest velocity backwards. Finds moment when z-velocity becomes insignificant.
            if (
                data["controller_right_vel.z"][i] <= -0.2
                and data["controller_right_vel.z"][i + 1] > -0.2
            ):
                end = i + 1
                break

    else:
        for i in range(
            pos, len(data["controller_right_vel.z"]) - 1
        ):  # Search forwards from moment of highest velocity forwards. Finds moment when z-velocity becomes insignificant.
            if (
                data["controller_right_vel.z"][i] >= 0.2
                and data["controller_right_vel.z"][i + 1] < 0.2
            ):
                end = i + 1
                break

    return (start, end)

In [313]:
filepath = "../Data/Combined/SRV_85.csv"
data = pd.read_csv(filepath, index_col=False)

prediction = simple_stat_classifier_2(filepath)

print(prediction)

get_start_end_after_classification(data, prediction)

SRV


(73, 124)

In [314]:
print(data['controller_right_vel.z'].to_string())

0     -0.185805
1     -0.163745
2     -0.127609
3     -0.074936
4     -0.015909
5      0.058024
6      0.123552
7      0.194110
8      0.284671
9      0.392398
10     0.498256
11     0.591574
12     0.671458
13     0.733913
14     0.767198
15     0.801241
16     0.816092
17     0.820441
18     0.804614
19     0.762595
20     0.695290
21     0.611684
22     0.528847
23     0.414515
24     0.254286
25     0.104777
26    -0.084616
27    -0.280297
28    -0.476775
29    -0.681022
30    -0.898144
31    -1.154065
32    -1.362952
33    -1.530531
34    -1.660454
35    -1.776302
36    -1.893140
37    -2.014106
38    -2.126556
39    -2.193530
40    -2.242550
41    -2.273883
42    -2.293757
43    -2.283906
44    -2.226465
45    -2.126550
46    -2.009040
47    -1.877243
48    -1.749099
49    -1.602072
50    -1.450269
51    -1.311684
52    -1.192594
53    -1.091935
54    -0.985491
55    -0.891710
56    -0.820332
57    -0.752199
58    -0.691649
59    -0.623346
60    -0.566401
61    -0.531341
62    -0